In [3]:
import pymysql

# Establishing the connection
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="Gowtham@#123"
)

# Creating a cursor object
cursor = conn.cursor()

# Creating a new database
cursor.execute("CREATE DATABASE IF NOT EXISTS youtube;")

print("Database created successfully!")

# Selecting the database
cursor.execute("USE youtube;")




Database created successfully!


0

In [4]:
# Create fact_content table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS fact_content (
        headline_id INT PRIMARY KEY AUTO_INCREMENT,
        headline VARCHAR(500),
        date_id INT,
        category_id INT,
        engagement_id INT
    );
""")

# Create dim_engagement table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dim_engagement (
        engagement_id INT PRIMARY KEY AUTO_INCREMENT,
        headline_id INT,
        date_id INT,
        views BIGINT,
        likes BIGINT,
        comments INT,
        engagement_rate FLOAT
    );
""")

# Create dim_date table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dim_date (
        date_id INT PRIMARY KEY AUTO_INCREMENT,
        headline_id INT,
        publish_date DATETIME,
        day INT,
        month INT,
        year INT,
        day_of_week VARCHAR(20)
    );
""")

# Create dim_content table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dim_content (
        category_id INT PRIMARY KEY AUTO_INCREMENT,
        headline_id INT,
        grouped_category VARCHAR(100)
    );
""")


print("Tables created successfully!")



Tables created successfully!


In [5]:
cursor.execute("ALTER TABLE fact_content MODIFY COLUMN headline TEXT;")
print("Modified headline column to TEXT!")

Modified headline column to TEXT!


In [6]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

df = pd.read_csv("youtube_data_sql.csv")

# Insert data into fact_content
for _, row in df.iterrows():
    cursor.execute("INSERT INTO fact_content (headline) VALUES (%s)", (row["headline"],))
    headline_id = cursor.lastrowid  # Get the last inserted ID

    # Insert into dim_date
    cursor.execute("""
        INSERT INTO dim_date (headline_id, publish_date, day, month, year, day_of_week) 
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (headline_id, row["date"], row["day"], row["month"], row["year"], row["day_of_week"]))

    date_id = cursor.lastrowid  # Get last inserted ID

    # Insert into dim_engagement
    cursor.execute("""
        INSERT INTO dim_engagement (headline_id, date_id, views, likes, comments, engagement_rate) 
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (headline_id, date_id, row["views"], row["likes"], row["comments"], row["engagement_rate"]))

    engagement_id = cursor.lastrowid  # Get last inserted ID

    # Insert into dim_content
    cursor.execute("""
        INSERT INTO dim_content (headline_id, grouped_category) VALUES (%s, %s)
    """, (headline_id, row["grouped_category"]))

    category_id = cursor.lastrowid  # Get last inserted ID

    # Update fact_content with IDs
    cursor.execute("""
        UPDATE fact_content 
        SET date_id = %s, category_id = %s, engagement_id = %s 
        WHERE headline_id = %s
    """, (date_id, category_id, engagement_id, headline_id))


print("Data inserted successfully!")



Data inserted successfully!


In [8]:
# Reconnect to MySQL if needed
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="Gowtham@#123",
    database="youtube"
)
cursor = conn.cursor()

# Run the query to check existing foreign keys
cursor.execute("""
    SELECT TABLE_NAME, CONSTRAINT_NAME 
    FROM information_schema.TABLE_CONSTRAINTS 
    WHERE TABLE_NAME IN ('fact_content', 'dim_engagement') 
    AND CONSTRAINT_TYPE = 'FOREIGN KEY';
""")

# Fetch results
foreign_keys = cursor.fetchall()

# Close connection

# Print foreign keys
if foreign_keys:
    print("Existing Foreign Keys:")
    for fk in foreign_keys:
        print(fk)
else:
    print("No foreign keys found in 'fact_content' or 'dim_engagement'.")


Existing Foreign Keys:
('dim_engagement', 'fk_engagement_date')
('fact_content', 'fk_category')
('fact_content', 'fk_date')
('fact_content', 'fk_engagement')


In [9]:
cursor.execute("""
    SELECT f.headline, d.publish_date, c.grouped_category, e.views, e.engagement_rate
    FROM fact_content f
    JOIN dim_date d ON f.date_id = d.date_id
    JOIN dim_content c ON f.category_id = c.category_id
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    LIMIT 5;
""")

for row in cursor.fetchall():
    print(row)


('Can Fit in Your Closet...', datetime.datetime(2025, 3, 13, 0, 0), 'Science & Technology', 273650, 0.0190682)
('Massage Your Mouth...', datetime.datetime(2025, 3, 5, 0, 0), 'Science & Technology', 917282, 0.00642878)
('The Ultimate Limited Edition Gaming Phone', datetime.datetime(2025, 2, 27, 0, 0), 'Science & Technology', 1001785, 0.0241289)
('The Unbelievably Thin Folding Smartphone...', datetime.datetime(2025, 2, 25, 0, 0), 'Science & Technology', 1194032, 0.0216091)
('Astonishing New Music Gadget...', datetime.datetime(2025, 2, 23, 0, 0), 'Science & Technology', 981959, 0.0184285)


In [10]:
# check Duplicate Records

cursor.execute("""
    SELECT headline, COUNT(*) AS count
    FROM fact_content
    GROUP BY headline
    HAVING count > 1;
""")
df_duplicates = pd.DataFrame(cursor.fetchall(), columns=["Headline", "Count"])
print(df_duplicates)


                                                 Headline  Count
0                               Can Fit in Your Closet...      4
1                                   Massage Your Mouth...      4
2               The Ultimate Limited Edition Gaming Phone      4
3             The Unbelievably Thin Folding Smartphone...      4
4                         Astonishing New Music Gadget...      4
...                                                   ...    ...
103282  Sundance, Ice-T, and Shades of the American Ra...      2
103283                      Good Games -- Is It possible?      2
103284    Google+ Now Open for Teens With Some Safeguards      2
103285                                           Web Wars      2
103286  RIM CEO Thorsten Heins' 'Significant' Plans Fo...      2

[103287 rows x 2 columns]


In [11]:
#Get Top 5 Most Engaging Headlines

cursor.execute("""
    SELECT f.headline, e.engagement_rate
    FROM fact_content f
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    ORDER BY e.engagement_rate DESC
    LIMIT 5;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Headline", "Engagement Rate"])
print(df)


                                            Headline  Engagement Rate
0  To the fans who've been there every step of th...         0.179591
1  To the fans who've been there every step of th...         0.179591
2  To the fans who've been there every step of th...         0.179591
3  To the fans who've been there every step of th...         0.179591
4  Happiest Birthday to the #OG🔥 Yo Yo Honey Sing...         0.168407


In [12]:
#Check unique Headlines 
cursor.execute("""
    SELECT DISTINCT f.headline, MAX(e.engagement_rate) AS engagement_rate
    FROM fact_content f
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    GROUP BY f.headline
    ORDER BY engagement_rate DESC
    LIMIT 5;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Headline", "Engagement Rate"])
print(df)


                                            Headline  Engagement Rate
0  To the fans who've been there every step of th...         0.179591
1  Happiest Birthday to the #OG🔥 Yo Yo Honey Sing...         0.168407
2                                    We are Groot 🥲🌿         0.167266
3         Spoiler Alert: You are part of a community         0.164120
4                     The Classism of Travel Culture         0.159713


In [13]:
#Total View and Likes based on Category

cursor.execute("""
    SELECT c.grouped_category, SUM(e.views) AS total_views, SUM(e.likes) AS total_likes
    FROM fact_content f
    JOIN dim_content c ON f.category_id = c.category_id
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    GROUP BY c.grouped_category
    ORDER BY total_views DESC;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Category", "Total Views", "Total Likes"])
print(df)


               Category  Total Views Total Likes
0       Health & Sports  17044712360   385048960
1    Lifestyle & Family   9052651162   361175478
2       News & Politics   6044527112   136875632
3    Business & Science   5311554144   119990784
4  Science & Technology   4885802968   109565542
5         Entertainment   4345009360   241673360
6             Education    149983128     5832076


In [24]:
#Total Views based on each month
cursor.execute("""
    SELECT d.year, d.month, COUNT(f.headline) AS total_articles, SUM(e.views) AS total_views
    FROM fact_content f
    JOIN dim_date d ON f.date_id = d.date_id
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    GROUP BY d.year, d.month
    ORDER BY d.year DESC, d.month DESC;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Year", "Month", "Total Videos&News", "Total Views"])
print(df)


     Year  Month  Total Videos&News Total Views
0    2025      3               1648   348067356
1    2025      2                804   412317940
2    2025      1                488   410325004
3    2024     12                272   231977624
4    2024     11                236   411042176
..    ...    ...                ...         ...
154  2012      5               2828   812037836
155  2012      4               2944   857073140
156  2012      3               3036   905025582
157  2012      2               2968   874526252
158  2012      1                316    97842612

[159 rows x 4 columns]


In [16]:
#The Most Liked Headlines
cursor.execute("""
    SELECT DISTINCT f.headline, e.likes
    FROM fact_content f
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    ORDER BY e.likes DESC
    LIMIT 10;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Headline", "Total Likes"])
print(df)


                                            Headline  Total Likes
0              The Smallest Microscope in the World!       738559
1  Shopping List (Official Video): Leo Grewal | Y...       491753
2                                Can you hear this!?       470914
3      I learned a system for remembering everything       411811
4                     This Gadget BRICKS Your Phone!       409056
5  I took cold showers for 30 days. Here's what I...       370991
6                                 My last video here       357101
7            Fastest Remote Control Car in the WORLD       355197
8                  The Most CUSTOMISABLE Phone Case!       296016
9                   Controlling a Bird With My Phone       276681


In [17]:
#Engagement Rate Comparison Between Categories
cursor.execute("""
    SELECT c.grouped_category, AVG(e.engagement_rate) AS avg_engagement_rate
    FROM fact_content f
    JOIN dim_content c ON f.category_id = c.category_id
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    GROUP BY c.grouped_category
    ORDER BY avg_engagement_rate DESC;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Category", "Average Engagement Rate"])
print(df)


               Category  Average Engagement Rate
0         Entertainment                 0.059293
1    Lifestyle & Family                 0.041792
2             Education                 0.038536
3       News & Politics                 0.025703
4       Health & Sports                 0.024082
5    Business & Science                 0.024082
6  Science & Technology                 0.021824


In [19]:
#The Most Commented Headlines
cursor.execute("""
    SELECT DISTINCT f.headline, e.comments
    FROM fact_content f
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    ORDER BY e.comments DESC
    LIMIT 10;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Headline", "Total Comments"])
print(df)


                                            Headline  Total Comments
0  Shopping List (Official Video): Leo Grewal | Y...           30129
1                                 My last video here           25379
2          How Trump’s second term will be different           20151
3  How China’s New AI Model DeepSeek Is Threateni...           17574
4                Why Google Search is Falling Apart.           16250
5                                Can you hear this!?           15316
6  How Companies Are Dodging Trump Tariffs On Can...           12850
7                           I bought my Dream House!           11547
8           Samsung S25 Ultra Hands On - The New Era           11103
9                      The BEST Smartphones of 2024!           11071


In [20]:
#The Most Popular Days for Content
cursor.execute("""
    SELECT d.day_of_week, COUNT(*) AS total_posts
    FROM fact_content f
    JOIN dim_date d ON f.date_id = d.date_id
    GROUP BY d.day_of_week
    ORDER BY total_posts DESC;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Day of Week", "Total Posts"])
print(df)


  Day of Week  Total Posts
0   Wednesday        47078
1     Tuesday        46978
2    Thursday        46444
3      Friday        45042
4      Monday        44072
5    Saturday        29918
6      Sunday        28460


In [23]:
#Monthly Content Performance Trends
cursor.execute("""
    SELECT d.year, d.month, COUNT(f.headline) AS total_articles, SUM(e.views) AS total_views, SUM(e.likes) AS total_likes
    FROM fact_content f
    JOIN dim_date d ON f.date_id = d.date_id
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    GROUP BY d.year, d.month
    ORDER BY d.year DESC, d.month DESC;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Year", "Month", "Total Videos&News", "Total Views", "Total Likes"])
print(df)


     Year  Month  Total Videos&News Total Views Total Likes
0    2025      3               1648   348067356    13648972
1    2025      2                804   412317940    16967108
2    2025      1                488   410325004    13614360
3    2024     12                272   231977624     7839388
4    2024     11                236   411042176    16965232
..    ...    ...                ...         ...         ...
154  2012      5               2828   812037836    23972278
155  2012      4               2944   857073140    25463626
156  2012      3               3036   905025582    26703404
157  2012      2               2968   874526252    27258362
158  2012      1                316    97842612     2939012

[159 rows x 5 columns]


In [26]:
#Viral Content (Views > 1M & High Engagement)
cursor.execute("""
    SELECT DISTINCT f.headline, e.views, e.engagement_rate
    FROM fact_content f
    JOIN dim_engagement e ON f.engagement_id = e.engagement_id
    WHERE e.views > 1000000 AND e.engagement_rate > 0.02
    ORDER BY e.views DESC;
""")
df = pd.DataFrame(cursor.fetchall(), columns=["Headline", "Total Views", "Engagement Rate"])
print(df)


                                              Headline  Total Views  \
0    Shopping List (Official Video): Leo Grewal | Y...     13520151   
1                The Smallest Microscope in the World!     13444093   
2                I tested the Craziest Xiaomi Gadgets!     10644064   
3        I learned a system for remembering everything      9479654   
4                                  Can you hear this!?      6764678   
..                                                 ...          ...   
118  MEAL PREP | 10 healthy meals in 1 HOUR (+ PDF ...      1047710   
119               Micro habits that improved my life ✨      1043287   
120  OnlyF*** model gets backlash over her $500k Fe...      1021294   
121      What’s the best way to treat the common cold?      1004834   
122          The Ultimate Limited Edition Gaming Phone      1001785   

     Engagement Rate  
0           0.038600  
1           0.055103  
2           0.026721  
3           0.043963  
4           0.071878  
..       

In [28]:
pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - --------------------

In [29]:
pip install matplotlib seaborn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.1 MB 1.6 MB/s eta 0:00:05
   ----- ---------------------------------- 1.0/8.1 MB 1.5 MB/s eta 0:00:05
   ----- ---------------------------------- 1.0/8.1 MB 1.5 MB/s eta 0:00:05
   ------ --------------------------------- 1.3/8.1 MB 1.3 MB/s eta 0:00:06
   ------ --------------------------------- 1.3/8.1 MB 1.3 MB/s eta 0:00:06
   ------- -------------------------------- 1.6/8.1 MB 964.5 kB/s eta 0:00:07
   ------- -------------------------------- 1.6/8.1 MB 964.5 kB/s eta 0:00:07
   ------- -------------------------------- 1.6/8.1 MB 964.5 kB/s eta 0:00:07
   ------- -------------------------------- 1.6/8.1 MB 964.5 kB/s eta 0:00:07
   ------- -------------------------------- 1.6/8.1 MB 964.5 kB/s eta 0:00:07
   --------- -

In [30]:
# Create a new Python file and write basic Streamlit code
with open("app.py", "w") as f:
    f.write("""import streamlit as st\nst.title('Hello, Streamlit!')""")

print("File 'app.py' created successfully!")


File 'app.py' created successfully!


In [ ]:
!streamlit run app.py
